In [10]:
import os
from Bio import SeqIO
from Bio.SeqUtils import CodonUsage
import pandas as pd
from collections import defaultdict

folder_path = 'C:/Users/seq'
filename = 'cds_from_genomic_thuringiensis.fna'
file_path = os.path.join(folder_path, filename)

def load_genome(file_path):
    sequences = []
    with open(file_path, 'r') as file:
        for record in SeqIO.parse(file, "fasta"):
            sequences.append(str(record.seq))
    return sequences

def calculate_codon_usage(sequences):
    codon_usage = defaultdict(int)
    total_codons = 0
    
    for sequence in sequences:
        for i in range(0, len(sequence) - 2, 3):
            codon = sequence[i:i+3]
            if len(codon) == 3:
                codon_usage[codon] += 1
                total_codons += 1

    for codon in codon_usage:
        codon_usage[codon] /= total_codons

    return pd.DataFrame.from_dict(codon_usage, orient='index', columns=['Relative Frequency'])

genome_sequences = load_genome(file_path)
codon_usage_df = calculate_codon_usage(genome_sequences)
print(codon_usage_df)

     Relative Frequency
ATG            0.027185
CAT            0.016198
CAA            0.030352
ACA            0.027369
TCT            0.015518
..                  ...
CCG            0.006812
GTC            0.005476
TAG            0.000598
CGG            0.001357
TGA            0.000599

[64 rows x 1 columns]


In [11]:
def design_optimized_gene(target_protein_seq, codon_usage_df):
    codon_table = {
        'A': ['GCT', 'GCC', 'GCA', 'GCG'],
        'C': ['TGT', 'TGC'],
        'D': ['GAT', 'GAC'],
        'E': ['GAA', 'GAG'],
        'F': ['TTT', 'TTC'],
        'G': ['GGT', 'GGC', 'GGA', 'GGG'],
        'H': ['CAT', 'CAC'],
        'I': ['ATT', 'ATC', 'ATA'],
        'K': ['AAA', 'AAG'],
        'L': ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'],
        'M': ['ATG'],
        'N': ['AAT', 'AAC'],
        'P': ['CCT', 'CCC', 'CCA', 'CCG'],
        'Q': ['CAA', 'CAG'],
        'R': ['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'],
        'S': ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'],
        'T': ['ACT', 'ACC', 'ACA', 'ACG'],
        'V': ['GTT', 'GTC', 'GTA', 'GTG'],
        'W': ['TGG'],
        'Y': ['TAT', 'TAC'],
        '*': ['TAA', 'TAG', 'TGA']
    }
    
    optimized_gene = ""
    for aa in target_protein_seq:
        possible_codons = codon_table[aa]
        best_codon = max(possible_codons, key=lambda codon: codon_usage_df.loc[codon, 'Relative Frequency'] if codon in codon_usage_df.index else 0)
        optimized_gene += best_codon
    
    return optimized_gene

target_protein_seq = "MVPQALLFVPLLVFPLCFGKFPIYTIPDKLGPWSPIDIHHLSCPNNLVVEDEGCTNLSGFSYMELKVGYISAIKMNGFTCTGVVTEAETYTNFVGYVTTTFKRKHFRPTPDACRAAYNWKMAGDPRYEESLHNPYPDYHWLRTVKTTKESLVIISPSVADLDPYDRSLHSRVFPGGNCSGVAVSSTYCSTNHDYTIWMPENPRLGMSCDIFTNSRGKRASKGSETCGFVDERGLYKSLKGACKLKLCGVLGLRLMDGTWVAMQTSNETKWCPPGQLVNLHDFRSDEIEHLVVEELVKKREECLDALESIMTTKSVSFRRLSHLRKLVPGFGKAYTIFNKTLMEADAHYKSVRTWNEIIPSKGCLRVGGRCHPHVNGVFFNGIILGPDGNVLIPEMQSSLLQQHMELLVSSVIPLMHPLADPSTVFKNGDEAEDFVEVHLPDVHERISGVDLGLPNWGKYVLLSAGALTALMLIIFLMTCWRRVNRSEPTQHNLRGTGREVSVTPQSGKIISSWESYKSGGETGL"

original_gene_seq = "ATGGTTCCTCAGGCTCTCCTGTTTGTACCCCTTCTGGTTTTTCCATTGTGTTTTGGGAAATTCCCTATTTACACGATACCAGACAAGCTTGGTCCCTGGAGCCCGATTGACATACATCACCTCAGCTGCCCAAACAATTTGGTAGTGGAGGACGAAGGATGCACCAACCTGTCAGGGTTCTCCTACATGGAACTTAAAGTTGGATACATCTCAGCCATAAAAATGAACGGGTTCACTTGCACAGGCGTTGTGACGGAGGCTGAAACCTACACTAACTTCGTTGGTTATGTCACAACCACGTTCAAAAGAAAGCATTTCCGCCCAACACCAGATGCATGTAGAGCCGCGTACAACTGGAAGATGGCCGGTGACCCCAGATATGAAGAGTCTCTACACAATCCGTACCCTGACTACCACTGGCTTCGAACTGTAAAAACCACCAAGGAGTCTCTCGTTATCATATCTCCAAGTGTGGCAGATTTGGACCCATATGACAGATCCCTTCACTCGAGGGTCTTCCCTGGCGGGAATTGCTCAGGAGTAGCGGTGTCTTCTACCTACTGCTCCACTAACCACGATTACACCATTTGGATGCCCGAGAATCCGAGACTAGGGATGTCTTGTGACATTTTTACCAATAGTAGAGGGAAGAGAGCATCCAAAGGGAGTGAGACTTGCGGCTTTGTAGATGAAAGAGGCCTATATAAGTCTTTAAAAGGAGCATGCAAACTCAAGTTATGTGGAGTTCTAGGACTTAGACTTATGGATGGAACATGGGTCGCGATGCAAACATCAAATGAAACCAAATGGTGCCCTCCCGGTCAGTTGGTGAATTTGCACGACTTTCGCTCAGACGAAATTGAGCACCTTGTTGTAGAGGAGTTGGTCAAGAAGAGAGAGGAGTGTCTGGATGCACTAGAGTCCATCATGACCACCAAGTCAGTGAGTTTCAGACGTCTCAGTCATTTAAGAAAACTTGTCCCTGGGTTTGGAAAAGCATATACCATATTCAACAAGACCTTGATGGAAGCCGATGCTCACTACAAGTCAGTCAGAACTTGGAATGAGATCATCCCTTCAAAAGGGTGTTTAAGAGTTGGGGGGAGGTGTCATCCTCATGTAAACGGGGTATTTTTCAATGGTATAATATTAGGACCTGACGGCAATGTCTTAATCCCAGAGATGCAATCATCCCTCCTCCAGCAACATATGGAGTTGTTGGTATCCTCGGTTATCCCCCTTATGCACCCCCTGGCAGACCCGTCTACCGTTTTCAAGAACGGTGACGAGGCTGAGGATTTTGTTGAAGTTCACCTTCCCGATGTGCACGAACGGATCTCAGGAGTTGACTTGGGTCTCCCGAACTGGGGGAAGTATGTATTACTGAGTGCAGGGGCCCTGACTGCCTTGATGTTGATAATTTTCCTGATGACATGCTGGAGAAGAGTCAATCGATCGGAACCTACACAACACAATCTCAGAGGGACAGGGAGGGAGGTGTCAGTCACTCCCCAAAGCGGGAAGATCATATCTTCATGGGAATCATACAAGAGCGGGGGTGAGACCGGACTGTGA"

optimized_gene_seq = design_optimized_gene(target_protein_seq, codon_usage_df)

highlighted_changes = ""
num_changed_codons = 0
num_same_codons = 0

for i in range(0, len(original_gene_seq), 3):
    original_codon = original_gene_seq[i:i+3]
    optimized_codon = optimized_gene_seq[i:i+3]
    if original_codon != optimized_codon:
        highlighted_changes += f"[{optimized_codon}]"
        num_changed_codons += 1
    else:
        highlighted_changes += optimized_codon
        num_same_codons += 1

print("Original Gene Sequence:  ", original_gene_seq)
print("Optimized Gene Sequence:  ", optimized_gene_seq)
print("Highlighted Changes:     ", highlighted_changes)
print(f"Number of changed codons: {num_changed_codons}")
print(f"Number of same codons: {num_same_codons}")


Original Gene Sequence:   ATGGTTCCTCAGGCTCTCCTGTTTGTACCCCTTCTGGTTTTTCCATTGTGTTTTGGGAAATTCCCTATTTACACGATACCAGACAAGCTTGGTCCCTGGAGCCCGATTGACATACATCACCTCAGCTGCCCAAACAATTTGGTAGTGGAGGACGAAGGATGCACCAACCTGTCAGGGTTCTCCTACATGGAACTTAAAGTTGGATACATCTCAGCCATAAAAATGAACGGGTTCACTTGCACAGGCGTTGTGACGGAGGCTGAAACCTACACTAACTTCGTTGGTTATGTCACAACCACGTTCAAAAGAAAGCATTTCCGCCCAACACCAGATGCATGTAGAGCCGCGTACAACTGGAAGATGGCCGGTGACCCCAGATATGAAGAGTCTCTACACAATCCGTACCCTGACTACCACTGGCTTCGAACTGTAAAAACCACCAAGGAGTCTCTCGTTATCATATCTCCAAGTGTGGCAGATTTGGACCCATATGACAGATCCCTTCACTCGAGGGTCTTCCCTGGCGGGAATTGCTCAGGAGTAGCGGTGTCTTCTACCTACTGCTCCACTAACCACGATTACACCATTTGGATGCCCGAGAATCCGAGACTAGGGATGTCTTGTGACATTTTTACCAATAGTAGAGGGAAGAGAGCATCCAAAGGGAGTGAGACTTGCGGCTTTGTAGATGAAAGAGGCCTATATAAGTCTTTAAAAGGAGCATGCAAACTCAAGTTATGTGGAGTTCTAGGACTTAGACTTATGGATGGAACATGGGTCGCGATGCAAACATCAAATGAAACCAAATGGTGCCCTCCCGGTCAGTTGGTGAATTTGCACGACTTTCGCTCAGACGAAATTGAGCACCTTGTTGTAGAGGAGTTGGTCAAGAAGAGAGAGGAGTGTCTGGATGCACTAGAGTCCATCATGACCACCAAGTCAGTGAGTTTCAGACGTCTCAGTCATTTAAGAAA

In [12]:
def cg_ratio(sequence):
    return (sequence.count('C') + sequence.count('G')) / len(sequence)

original_cg_ratio = cg_ratio(original_gene_seq)
optimized_cg_ratio = cg_ratio(optimized_gene_seq)

print(f"Oryginalny stosunek par CG: {original_cg_ratio:.2f}")
print(f"Zoptymalizowany stosunek par CG: {optimized_cg_ratio:.2f}")

Oryginalny stosunek par CG: 0.48
Zoptymalizowany stosunek par CG: 0.31
